#### **This is the vector search function exploration**

In [1]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore(OpenAIEmbeddings())

In [18]:
from langchain_core.documents import Document

document_1 = Document(id="1", 
                      page_content="foo", 
                      metadata={"baz": "baz"}
                     )
document_2 = Document(id="2", 
                      page_content="thud", 
                      metadata={"bar": "baz"}
                     )
document_3 = Document(id="3", 
                      page_content="i will be deleted :(",
                      metadata={"bar": "baz"}
                     )

documents = [document_1, document_2, document_3]
vector_store.add_documents(documents=documents)

['1', '2', '3']

In [19]:
top_n = 10
for index, (id, doc) in enumerate(vector_store.store.items()):
    print(index, id, doc.keys())
    print(doc['text'])

0 1 dict_keys(['id', 'vector', 'text', 'metadata'])
foo
1 2 dict_keys(['id', 'vector', 'text', 'metadata'])
thud
2 3 dict_keys(['id', 'vector', 'text', 'metadata'])
i will be deleted :(


In [20]:
results = vector_store.similarity_search(query="thud", k=2)
for doc in results:
    print(doc)
    print(f"* {doc.page_content} [{doc.metadata}]")

page_content='thud' metadata={'bar': 'baz'}
* thud [{'bar': 'baz'}]
page_content='foo' metadata={'baz': 'baz'}
* foo [{'baz': 'baz'}]


#### **Search with filter**

In [21]:
def _filter_function(doc: Document) -> bool:
    return doc.metadata.get("bar") == "baz"

results = vector_store.similarity_search(
    query="thud", k=2, filter=_filter_function
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* thud [{'bar': 'baz'}]
* i will be deleted :( [{'bar': 'baz'}]


#### **Search with score**

In [24]:
results = vector_store.similarity_search_with_score(
    query="thug",
    k=1
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.855233] thud [{'bar': 'baz'}]


#### **Async**

In [26]:
# search with score
results = await vector_store.asimilarity_search_with_score(query="qux", k=1)
for doc,score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.832237] foo [{'baz': 'baz'}]
